## Loan Purchase dataset: Feature Selection

In the following cells, we will select a group of variables, the most predictive ones, to build our machine learning models. 

In [59]:
# to handle datasets
import pandas as pd
import numpy as np
import os

# for plotting
import matplotlib.pyplot as plt
%matplotlib inline

# to build the models
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [8]:
# load dataset

# Set Directory
os.chdir('C:/Users/aksha/Desktop/BOI_Challenge/')

X_train = pd.read_csv('xtrain.csv')
X_test = pd.read_csv('xtest.csv')

X_train.head()

,Client_ID,Prod_Held_Count,Prv_Loan_Flag,Num_txns,Lst_txn_amt,Loan_Flag,Gender_lbl,Province_lbl,Income_Grp_lbl,Age_Grp_lbl,Avg_CA_txn_amt_grp_lbl
0,1,4,1,0.00,0.498974,0,1,2,2,2,1
1,2,4,0,0.17,0.087504,0,1,4,1,2,3
2,3,2,0,0.25,0.582233,0,0,1,2,2,1
3,4,2,1,0.13,0.072993,0,1,1,3,3,1
4,5,1,0,0.39,0.283609,0,0,1,4,4,1


In [18]:
# capture the target
y_train = X_train['Loan_Flag']
y_test = X_test['Loan_Flag']

# drop unnecessary variables from our training and testing sets
X_train.drop(['Client_ID', 'Loan_Flag'], axis=1, inplace=True)
X_test.drop(['Client_ID'], axis=1, inplace=True)

**IMP**: Lasso Regression is used to cross verify the features selected from WOE and IV.

In [28]:
# here I will do the model fitting and feature selection
# altogether in one line of code

# first, I specify the Lasso Regression model, and I
# select a suitable alpha (equivalent of penalty).
# The bigger the alpha the less features that will be selected.

# Then I use the selectFromModel object from sklearn, which
# will select the features which coefficients are non-zero

sel_ = SelectFromModel(Lasso(alpha=0.002, random_state=0)) # remember to set the seed, the random state in this function
sel_.fit(X_train, y_train)

SelectFromModel(estimator=Lasso(alpha=0.002, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=0,
   selection='cyclic', tol=0.0001, warm_start=False),
        max_features=None, norm_order=1, prefit=False, threshold=None)

In [29]:
# this command let's us visualise those features that were kept.
# Kept features have a True indicator
sel_.get_support()

array([False,  True,  True, False, False, False, False,  True,  True])

In [30]:
# let's print the number of total and selected features

# this is how we can make a list of the selected features
selected_feat = X_train.columns[(sel_.get_support())]

# let's print some stats
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(sel_.estimator_.coef_ == 0)))

total features: 9
selected features: 4
features with coefficients shrank to zero: 5


In [31]:
# print the selected features
selected_feat

Index(['Prv_Loan_Flag', 'Num_txns', 'Age_Grp_lbl', 'Avg_CA_txn_amt_grp_lbl'], dtype='object')

In [25]:
#Identify the selected variables

In [26]:
# this is an alternative way of identifying the selected features 
# based on the non-zero regularisation coefficients:
selected_feats = X_train.columns[(sel_.estimator_.coef_ != 0).ravel().tolist()]
selected_feats

Index(['Prv_Loan_Flag', 'Num_txns', 'Avg_CA_txn_amt_grp_lbl'], dtype='object')

In [13]:
df = X_train.drop(['Client_ID'], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 10 columns):
Prod_Held_Count           9914 non-null int64
Prv_Loan_Flag             9914 non-null int64
Num_txns                  9914 non-null float64
Lst_txn_amt               9914 non-null float64
Loan_Flag                 9914 non-null int64
Gender_lbl                9914 non-null int64
Province_lbl              9914 non-null int64
Income_Grp_lbl            9914 non-null int64
Age_Grp_lbl               9914 non-null int64
Avg_CA_txn_amt_grp_lbl    9914 non-null int64
dtypes: float64(2), int64(8)
memory usage: 774.6 KB


**Fine Classing and Coarse Classing**

This approach is used to select high predictor variables having high cardinality using Weight of Evidence and Information Value.

In [10]:
# import packages
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [71]:
final_iv, IV = data_vars(df,df.Loan_Flag)

In [72]:
final_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,Prod_Held_Count,1.000000,2.000000,3955,70,0.017699,3885,0.982301,0.429448,0.398421,0.074992,0.019548
1,Prod_Held_Count,3.000000,4.000000,3986,69,0.017311,3917,0.982689,0.423313,0.401702,0.052400,0.019548
2,Prod_Held_Count,5.000000,5.000000,1973,24,0.012164,1949,0.987836,0.147239,0.199877,-0.305643,0.019548
3,Prv_Loan_Flag,0.000000,0.000000,7442,26,0.003494,7416,0.996506,0.159509,0.760537,-1.561924,1.693404
4,Prv_Loan_Flag,1.000000,1.000000,2472,137,0.055421,2335,0.944579,0.840491,0.239463,1.255589,1.693404
5,Num_txns,0.000000,0.010000,3528,1,0.000283,3527,0.999717,0.006135,0.361706,-4.076828,3.042332
6,Num_txns,0.020000,0.230000,3117,3,0.000962,3114,0.999038,0.018405,0.319352,-2.853676,3.042332
7,Num_txns,0.240000,1.000000,3269,159,0.048639,3110,0.951361,0.975460,0.318942,1.117901,3.042332
8,Lst_txn_amt,0.000000,0.498974,6590,71,0.010774,6519,0.989226,0.435583,0.668547,-0.428421,0.223815
9,Lst_txn_amt,0.499268,1.000000,3324,92,0.027677,3232,0.972323,0.564417,0.331453,0.532307,0.223815


In [76]:
IV.sort_values('IV')

,VAR_NAME,IV
2,Gender_lbl,0.000162
3,Income_Grp_lbl,0.001496
7,Province_lbl,0.003200
0,Age_Grp_lbl,0.003876
6,Prod_Held_Count,0.019548
4,Lst_txn_amt,0.223815
1,Avg_CA_txn_amt_grp_lbl,0.447791
8,Prv_Loan_Flag,1.693404
5,Num_txns,3.042332


From the above Information Value (IV) the following features are found to be good predictors have a strong relationship with loan purchasability:

      Variable               IV Value
      
1. Prod_Held_Count-----------0.019548
2. Lst_txn_amt---------------0.223815
3. Avg_CA_txn_amt_grp_lbl----0.447791

The below features are found to be too good to be true.

      Variable               IV Value
      
1. Prv_Loan_Flag-------------1.693404
2. Num_txns------------------3.042332

In [82]:
slctd_feat_IV_WOE = IV[IV['IV'].between(0.005, 0.5, inclusive=False)]
# let's print some stats
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(slctd_feat_IV_WOE)))
print('The features and IV values are as follows:')
slctd_feat_IV_WOE

total features: 9
selected features: 3
The features and IV values are as follows:


,VAR_NAME,IV
1,Avg_CA_txn_amt_grp_lbl,0.447791
4,Lst_txn_amt,0.223815
6,Prod_Held_Count,0.019548


In [81]:
# now we save the selected list of features
pd.Series(slctd_feat_IV_WOE["VAR_NAME"]).to_csv('slctd_feat_IV_WOE.csv', index=False)

**Please continue to BOI_Challenge_Model_Build.ipynb**